In [ ]:
using LinearAlgebra
using JuMP, Ipopt
using Plots
using BSON: @save @load

In [ ]:
# load dataset
@load "vineQVF.bson" Q V F
numPins, numTimesteps = size(Q)

In [ ]:
# set up for optimization
A = [Q' V']
A = kron(A,Matrix(I,numPins,numPins))
numParams = 2*numPins^2

y = vec(F)

# weights matrix for regularization
W = I

In [ ]:
# solve optimization
model = Model(optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
set_silent(model)
@variable(model, x[1:numParams])
@objective(model, Min, (A*x-y)'*(A*x-y) + x'*W*x)
JuMP.optimize!(model)

# extract results
params = JuMP.value.(x)
KCmatrix = reshape(params, numPins, 2*numPins)

In [ ]:
# show K
K = KCmatrix[:, 1:numPins]
heatmap(K)

In [ ]:
# show C
C = KCmatrix[:, numPins+1:end]
heatmap(C)

In [ ]:
# compute goodness of fit
SSR = (A*params-y)'*(A*params-y)
SST = norm(y .- mean(y))^2
Rsq = 1.0 - SSR/SST